# TMB split 만들때 subtyping까지 고려해서 만들도록
이미 subtype 별로 TMB high, low balance가 맞아서 별로 효과 없었음

In [30]:
import pdb
import os
# os.chdir('/shared/j.jang/pathai/CLAM')
import pandas as pd
from datasets.dataset_generic import Generic_WSI_Classification_Dataset, Generic_MIL_Dataset, save_splits
import numpy as np
# from utils.utils import generate_split, nth
import math
from sklearn.model_selection import StratifiedShuffleSplit

csv_path = '/shared/js.yun/CLAM/dataset_csv/TCGA-lung-LUAD+LUSC-TMB-pan_cancer-323_HIPT_only.csv'
# split_dir = '/shared/js.yun/data/CLAM_data/TCGA-lung-luad+lusc-TMB-323-HIPT-only-splits/'
split_dir = '/shared/js.yun/data/CLAM_data/test/'
label_dict = {"TMB_low":0, "TMB_high":1}

task = 'task_1_tumor_vs_normal'
seed = 1
label_frac = 1.0
val_frac = 0.1
test_frac = 0.2
k = 1
n_class = 2

os.makedirs(split_dir, exist_ok=True)

class TMB_Dataset(Generic_WSI_Classification_Dataset):
    def __init__(self,
        subtype_head = 'Oncotree Code', 
        **kwargs):
        super(TMB_Dataset, self).__init__(**kwargs)

        self.subtype_col = self.slide_data[subtype_head]
        self.subtype_classes = np.unique(self.subtype_col)
        self.tmb_count_group_by_subtype = self.slide_data.groupby('label')[self.subtype_col.name].value_counts().unstack().fillna(0)
        print(self.tmb_count_group_by_subtype)
        # print(self.subtype_col.value_counts())
        # print(self.slide_data.groupby('label')[self.subtype_col.name].value_counts().unstack().fillna(0))
        
        
        num_slides_cls = np.array([len(cls_ids) for cls_ids in dataset.patient_cls_ids])
        val_num = np.round(num_slides_cls * val_frac).astype(int)
        test_num = np.round(num_slides_cls * test_frac).astype(int)
        dataset.create_splits(k = k, val_num = val_num, test_num = test_num)

        self.split_gen = self.stratified_split(0.7, 0.1, 0.2)

    def create_splits(self, k = 3, val_num = (25, 25), test_num = (40, 40), label_frac = 1.0, custom_test_ids = None):
        '''
        얘는 딱히 고칠거 없음
        self.split_gen에다가 generate_split 붙여주는 역할
        '''
        settings = {
                    'n_splits' : k, 
                    'val_num' : val_num, 
                    'test_num': test_num,
                    'label_frac': label_frac,
                    'seed': self.seed,
                    'custom_test_ids': custom_test_ids
                    }

        if self.patient_strat:
            settings.update({'cls_ids' : self.patient_cls_ids, 'samples': len(self.patient_data['case_id'])})
        else:
            settings.update({'cls_ids' : self.slide_cls_ids, 'samples': len(self.slide_data)})

        # self.split_gen = self.generate_split(**settings)

    def stratified_split(self, a, b, c):
        # Create a combined stratification column
        self.slide_data['strata'] = self.slide_data['label'].astype(str) + "_" + self.subtype_col.astype(str)

        # Calculate the number of splits based on the provided ratios
        test_size = c / (a + b + c)
        val_size = b / (a + b)  # Adjusted this line

        # Create the stratified splitter for train+val and test
        sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=42)
        for train_val_index, test_index in sss.split(self.slide_data, self.slide_data['strata']):
            pass

        # Create the stratified splitter for train and val within train+val
        sss_val = StratifiedShuffleSplit(n_splits=1, test_size=val_size, random_state=42)
        for train_index, val_index in sss_val.split(self.slide_data.iloc[train_val_index], self.slide_data.iloc[train_val_index]['strata']):
            train_index = train_val_index[train_index]  # Adjust indices to original dataframe
            val_index = train_val_index[val_index]      # Adjust indices to original dataframe

        # Drop the strata column
        self.slide_data.drop(columns=['strata'], inplace=True)

        self.train_ids, self.val_ids, self.test_ids = train_index, val_index, test_index
        
        print(self.train_ids)
        print(self.val_ids)
        print(self.test_ids)

        # return train_index, val_index, test_index


    # def generate_split(cls_ids, val_num, test_num, samples, n_splits = 5,
    #     seed = 7, label_frac = 1.0, custom_test_ids = None):
    #     '''
    #     호출할때마다 (sampled_train_ids, all_val_ids, all_test_ids) yield
    #     '''
    #     indices = np.arange(samples).astype(int)
        
    #     if custom_test_ids is not None:
    #         indices = np.setdiff1d(indices, custom_test_ids)

    #     np.random.seed(seed)
    #     for i in range(n_splits):
    #         all_val_ids = []
    #         all_test_ids = []
    #         sampled_train_ids = []
            
    #         if custom_test_ids is not None: # pre-built test split, do not need to sample
    #             all_test_ids.extend(custom_test_ids)

    #         for c in range(len(val_num)):
    #             possible_indices = np.intersect1d(cls_ids[c], indices) #all indices of this class
    #             val_ids = np.random.choice(possible_indices, val_num[c], replace = False) # validation ids

    #             remaining_ids = np.setdiff1d(possible_indices, val_ids) #indices of this class left after validation
    #             all_val_ids.extend(val_ids)

    #             if custom_test_ids is None: # sample test split

    #                 test_ids = np.random.choice(remaining_ids, test_num[c], replace = False)
    #                 remaining_ids = np.setdiff1d(remaining_ids, test_ids)
    #                 all_test_ids.extend(test_ids)

    #             if label_frac == 1:
    #                 sampled_train_ids.extend(remaining_ids)
                
    #             else:
    #                 sample_num  = math.ceil(len(remaining_ids) * label_frac)
    #                 slice_ids = np.arange(sample_num)
    #                 sampled_train_ids.extend(remaining_ids[slice_ids])

    #         yield sampled_train_ids, all_val_ids, all_test_ids

if task == 'task_1_tumor_vs_normal':
    dataset = TMB_Dataset(subtype_head = 'Oncotree Code',
                            csv_path = csv_path,
                            shuffle = False, 
                            seed = seed, 
                            print_info = True,
                            label_dict = label_dict,
                            patient_strat=True,
                            ignore=[])
elif task == 'task_2_tumor_subtyping':
    dataset = Generic_WSI_Classification_Dataset(csv_path = csv_path,
                            shuffle = False, 
                            seed = seed, 
                            print_info = True,
                            label_dict = label_dict,
                            patient_strat= True,
                            patient_voting='maj',
                            ignore=[])
else:
    raise NotImplementedError

# train_data, val_data, test_data = dataset.stratified_split(0.6, 0.2, 0.2)
# print(train_data)
# print(val_data)
# print(test_data)


# num_slides_cls = np.array([len(cls_ids) for cls_ids in dataset.patient_cls_ids])
# val_num = np.round(num_slides_cls * val_frac).astype(int)
# test_num = np.round(num_slides_cls * test_frac).astype(int)


descriptor_df = dataset.test_split_gen(return_descriptor=True)
splits = dataset.return_splits(from_id=True)
save_splits(splits, ['train', 'val', 'test'], os.path.join(split_dir, 'splits_{}.csv'.format(0)))
save_splits(splits, ['train', 'val', 'test'], os.path.join(split_dir, 'splits_{}_bool.csv'.format(0)), boolean_style=True)
descriptor_df.to_csv(os.path.join(split_dir, 'splits_{}_descriptor.csv'.format(0)))

label column: label
label dictionary: {'TMB_low': 0, 'TMB_high': 1}
number of classes: 2
slide-level counts:  
 0    686
1    255
Name: label, dtype: int64
Patient-LVL; Number of samples registered in class 0: 686
Slide-LVL; Number of samples registered in class 0: 686
Patient-LVL; Number of samples registered in class 1: 255
Slide-LVL; Number of samples registered in class 1: 255
Oncotree Code  LUAD  LUSC
label                    
0               346   340
1               141   114
[720 704 330 140 193 384 154  74 288 321 445 884 216 761 917 386 185 759
 512 647 542  70 419 341 898 474 615 835  55 176 502 678 767 196 849 847
 563 817 276 780 715 489 923 166 722 101 107 183 586 723 494  81 646 313
 706 443  38 481 744 226 218 462 124 737 406  96  35 558 246 551 143 894
 820 475 245 679 673 868 559 731 688 550 129 567 499 802 786  37 581 232
 867 181 750 331 300 464   3 333 614 104 540 578 636 492 661 574 393 253
 685 160 467 272 472 168 319 468 602 539 746 860 146 580 572 911 486 590
 

In [2]:
if label_frac > 0:
    label_fracs = [label_frac]
else:
    label_fracs = [0.1, 0.25, 0.5, 0.75, 1.0]

for lf in label_fracs:
    split_dir = split_dir + str(task) + '_{}'.format(int(lf * 100))
    os.makedirs(split_dir, exist_ok=True)
    dataset.create_splits(k = k, val_num = val_num, test_num = test_num, label_frac=lf)
    for i in range(k):
        dataset.set_splits()
        descriptor_df = dataset.test_split_gen(return_descriptor=True)
        splits = dataset.return_splits(from_id=True)
        save_splits(splits, ['train', 'val', 'test'], os.path.join(split_dir, 'splits_{}.csv'.format(i)))
        save_splits(splits, ['train', 'val', 'test'], os.path.join(split_dir, 'splits_{}_bool.csv'.format(i)), boolean_style=True)
        descriptor_df.to_csv(os.path.join(split_dir, 'splits_{}_descriptor.csv'.format(i)))



# 밑에는 test 코드

In [16]:
class Parent:
    def say_hello(self):
        print("Hello from Parent")

class Child(Parent):
    def say_hello(self):
        super().say_hello()  # 부모 클래스의 메서드 호출
    def say_hello(self):
        print("Hello from Child")

# 객체 생성
child_obj = Child()

# 메서드 호출
child_obj.say_hello()



Hello from Child


In [9]:
label_dict = {'LUSC':0, 'LUAD':1}
num_classes = 2

index = [list(label_dict.keys())[list(label_dict.values()).index(i)] for i in range(num_classes)]
print(index)
print(list(label_dict.keys()))
print(list(label_dict.values()).index(1))


['LUSC', 'LUAD']
['LUSC', 'LUAD']
1
